## Build a full training set
#### split 512x512 to 64x64
#### remove background and blurred, using conv filter 

In [1]:
import os
import cv2
import numpy as np

In [2]:
rootpath="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\512_image_label_full"
#set the paths to save the 64x64 sutiles according to their labels.
path_64_pos="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image\\nor_full"
path_64_neg="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\training\\split64_image\\tum_full"

In [3]:
limgs = []
for root, folders, files in os.walk(rootpath, topdown=False):
    for name in files:
        if name.endswith('.tif'):
            limgs.append(root+'\\'+name)
# print(len(limgs))

In [4]:
llabels = []
for root, folders, files in os.walk(rootpath, topdown=False):
    for name in files:
        if name.endswith('.tiff'):
            llabels.append(root+'\\'+name)
# print(len(llabels))

In [5]:
kernel = np.array(([-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1]), np.float32)
threshold = 200
threshold_filter = 35

In [8]:
n5kn = 0
n5kp = 0
os.mkdir(os.path.join(path_64_neg, str(n5kn).zfill(3)))
os.mkdir(os.path.join(path_64_pos, str(n5kp).zfill(3)))

from tqdm import tqdm
pbar1 = tqdm(total=len(limgs), desc='512x512 patch', miniters=1, position=0)

for i in range(len(limgs)):
    pbar1.update()    
    img = cv2.cvtColor(cv2.imread(limgs[i], -1), cv2.COLOR_BGR2RGB)
    label = cv2.imread(llabels[i], -1)/255
    nsub = 0
    img_subtile = np.ndarray(shape=(64,64,64,3), dtype='uint8')
    label_subtile = np.ndarray(shape=(64,64,64)) #already resize to 0-1, if set dtype to uint8, everything will be 0

    for c in range(0, 511, 64):
        c_64 = c + 64
        for r in range(0, 511, 64):
            r_64 = r + 64
            img_subtile[nsub,:,:,:] = img[r:r_64,c:c_64,:]
            label_subtile[nsub,:,:] = label[r:r_64,c:c_64]
            nsub = nsub + 1

    for j in range(nsub):
        m_sub = np.mean(img_subtile[j,:,:,:])
        y = cv2.filter2D(img_subtile[j,:,:,:], -1, kernel)
        for a in range(y.shape[0]):
            for b in range(y.shape[1]):
                for c in range(y.shape[2]):
                    if y[a,b,c] < 0:
                        y[a,b,c] = 0
                    if y[a,b,c] > 255:
                        y[a,b,c] = 255
        m_filtered_sub = np.mean(y)

        if (m_sub < threshold) & (m_filtered_sub > threshold_filter):
            if sum(sum(label_subtile[j,:,:] == 3/255)) > 3072:
                nneg = len([x for x in os.listdir(os.path.join(path_64_neg, str(n5kn).zfill(3))) if x.endswith('.tif')])
                if (nneg==5000):
                    n5kn = n5kn + 1
                    nneg = 0
                    os.mkdir(os.path.join(path_64_neg, str(n5kn).zfill(3)))
                nneg = str(nneg).zfill(4)
                cv2.imwrite(os.path.join(path_64_neg, str(n5kn).zfill(3), 'tum_'+str(n5kn).zfill(3)+'_'+nneg+'.tif'), cv2.cvtColor(img_subtile[j,:,:,:], cv2.COLOR_RGB2BGR))
            elif sum(sum(label_subtile[j,:,:] == 2/255)) > 3072:
                npos = len([x for x in os.listdir(os.path.join(path_64_pos, str(n5kp).zfill(3))) if x.endswith('.tif')])
                if (npos==5000):
                    n5kp = n5kp + 1
                    npos = 0
                    os.mkdir(os.path.join(path_64_pos, str(n5kp).zfill(3)))
                npos = str(npos).zfill(4)
                cv2.imwrite(os.path.join(path_64_pos, str(n5kp).zfill(3), 'nor_'+str(n5kp).zfill(3)+'_'+npos+'.tif'), cv2.cvtColor(img_subtile[j,:,:,:], cv2.COLOR_RGB2BGR))

# print(limg)
# print(llabel)

512x512 patch: 100%|███████████████| 233016/233016 [380:31:03<00:00,  3.87s/it]

In [6]:
len(limgs)

233016